# Unlocking Langchain Framework

In [1]:
from MenuGenerator.secret_key import openapi_key
import os

os.environ['OPENAI_API_KEY'] = openapi_key


# LLM Practice

In [2]:
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0.7)
city_name = llm.predict("I want to open a Indian restaurant in UK, give me only the best city name for that.")
print(city_name)



The best city in the UK for opening an Indian restaurant is London. With its diverse population, plentiful tourist attractions, and vibrant nightlife, London is a great place to open a restaurant. Additionally, it is the most populous city in the UK, providing a large potential customer base.


In [4]:
llm("I want to open a Turkish restaurant in US, give me only the best city name for that.")

'\n\nChicago, Illinois.'

# Prompt Templates

In [5]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, give me only one city name for that."
)
prompt = prompt_template_name.format(culture="Arabic", location="UK")
print("prompt:",prompt)
print(llm.predict(prompt))

prompt: I want to open a Arabic restaurant in UK, give me only one city name for that.


Manchester


# Chains

In [6]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run(culture="Arabic", location="Turkey")

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-ISkY9czDa2pZEcFeMUOQyQT3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [7]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run(culture="Mexican", location="Italy")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a Mexican restaurant in Italy, give me only one city name for that.


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-ISkY9czDa2pZEcFeMUOQyQT3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [8]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables = ['location'],
    template = "I want to open a Turkish restaurant in {location}, give me only the best city name for that."
)
name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)
food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [9]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains=[name_chain, food_items_chain])

content = chain.run("Germany")
content

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-ISkY9czDa2pZEcFeMUOQyQT3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

#### Sequential Chain

In [22]:
llm = OpenAI(temperature=0.7)

prompt_template_city_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, Suggest a fancy city for this."
)

city_name_chain = LLMChain(llm=llm, prompt=prompt_template_city_name, output_key="city_name")

In [23]:
llm = OpenAI(temperature=0.7)

prompt_template_res_name = PromptTemplate(
    input_variables = ['city_name'],
    template = "Suggest a restaurant name in {city_name}."
)

retaurant_name_chain = LLMChain(llm=llm, prompt=prompt_template_res_name, output_key="restaurant_name")

In [24]:
llm = OpenAI(temperature=0.7)

prompt_template_menu_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_menu_items, output_key="menu_items")

In [25]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [city_name_chain, retaurant_name_chain, food_items_chain],
    input_variables = ['culture', 'location'],
    output_variables = ['city_name', 'restaurant_name', "menu_items"]
)

In [26]:
chain({"culture": "Indian", "location":"Norway"})

{'culture': 'Indian',
 'location': 'Norway',
 'city_name': '\n\nOslo is an ideal city to open an Indian restaurant in Norway. With its vibrant and diverse culture, Oslo is considered one of the most exciting cities in Scandinavia. The city boasts a number of excellent Indian restaurants, which are very popular with both locals and tourists alike. With its beautiful scenery and many attractions, Oslo is an ideal place to open a successful Indian restaurant.',
 'restaurant_name': '\n\nSuggested Restaurant Name: Spice of Oslo',
 'menu_items': '\n\nAppetizers\n- Crispy Egg Rolls with Soy Dipping Sauce\n- Sweet Potato Fritters with Avocado Salsa\n- Baked Garlic and Herb Focaccia\n- Grilled Asparagus with Lemon-Butter Sauce\n\nEntrees\n- Roasted Red Pepper Pasta with Artichoke Hearts\n- Coconut Curry Shrimp with Jasmine Rice\n- Grilled Salmon Fillet with Mango Salsa\n- Grilled Lamb Chops with Mint Chimichurri\n\nDesserts\n- Chocolate-Hazelnut Mousse Tart\n- Caramelized Pineapple with Coconut

## Agents

In [3]:
from langchain.agents import AgentType, initialize_agent, load_tools


### llm-math and wikipedia tools

In [33]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

agent.run("Who is the current leader of Japan? What is age of that leader right now?")



> Entering new AgentExecutor chain...
 I need to look up the current leader of Japan and then look up their age.
Action: Wikipedia
Action Input: current leader of Japan
Observation: Page: List of current heads of state and government
Summary: This is a list of current heads of state and heads of government. In some cases, mainly in presidential systems, there is only one leader being both head of state and head of government. In other cases, mainly in semi-presidential and parliamentary systems, the head of state and the head of government are different people. In semi-presidential and parliamentary systems, the head of government role (i.e. executive branch) is fulfilled by both the listed head of government and the head of state. In single-party systems, ruling party's leader (i.e. General Secretary) is usually the de facto top leader of the state, though sometimes this leader also holds the presidency or premiership.
The list includes the names of recently elected or appointed hea

'The current leader of Japan is Yoshihide Suga, and he is 67 years old.'

### Google Search API (serpapi) and llm-math

In [4]:
# make sure yoy have installed this package: pip install google-search-results
from MenuGenerator.secret_key import serpapi_key
os.environ['SERPAPI_API_KEY'] = serpapi_key

In [5]:
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What is the GDP of US in 2022 plus 5?")



> Entering new AgentExecutor chain...
 I need to find the GDP of US in 2022 and then add 5
Action: Search
Action Input: "GDP of US in 2022"
Observation: $25.46 trillion
Thought: I need to add 5 to this number
Action: Calculator
Action Input: 25.46 + 5
Observation: Answer: 30.46
Thought:

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for text-davinci-003 in organization org-ISkY9czDa2pZEcFeMUOQyQT3 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

### Memory

In [7]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


prompt_template_name = PromptTemplate(
    input_variables = ['culture', 'location'],
    template = "I want to open a {culture} restaurant in {location}, give me only one city name for that."
)

chain = LLMChain(llm=llm, prompt=prompt_template_name)
name = chain.run(culture="Arabic", location="UK")
print(name)



Birmingham


In [8]:
name = chain.run(culture="Indian", location="US")
print(name)



Chicago, Illinois.


In [9]:
chain.memory

In [10]:
type(chain.memory)

NoneType

In [15]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.run("Mexican")
print(name)



Taco Fiesta


In [16]:
name = chain.run("Arabic")
print(name)



Al-Fez Restaurant


In [17]:
chain.memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Mexican'), AIMessage(content='\n\nTaco Fiesta'), HumanMessage(content='Arabic'), AIMessage(content='\n\nAl-Fez Restaurant')]))

In [19]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Taco Fiesta
Human: Arabic
AI: 

Al-Fez Restaurant


#### Conversation Chains

In [20]:
from langchain.chains import ConversationChain

convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [21]:
convo.run("Who won the first cricket world cup?")

' The first cricket world cup was won by the West Indies in 1975. They defeated Australia by 17 runs in the final match.'

In [22]:
convo.run("How much is 5+5?")

' The answer to 5+5 is 10.'

In [23]:
convo.run("Who was the captain ofthe winning team?")

' The captain of the winning team in the first cricket world cup was Clive Lloyd.'

In [24]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975. They defeated Australia by 17 runs in the final match.
Human: How much is 5+5?
AI:  The answer to 5+5 is 10.
Human: Who was the captain ofthe winning team?
AI:  The captain of the winning team in the first cricket world cup was Clive Lloyd.


#### Conversation Buffer Window Memory

In [25]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

convo = ConversationChain(
    llm=OpenAI(temperature=0.7),
    memory=memory
)
convo.run("Who won the first cricket world cup?")

" The first cricket world cup was won by the West Indies in 1975. They beat Australia by 17 runs in the final match at Lord's in London."

In [26]:
convo.run("How much is 5+5?")

' 5+5 is 10.'

In [27]:
convo.run("Who was the captain of the winning team?")

' I do not know the answer to that question.'